In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.neural_network import MLPRegressor

In [2]:
df=pd.read_csv('/home/jolivera/Documents/CloudSkin/Time-Series-Library/ccgrid_scripts/data/training_data_scored_pairs.csv')

In [16]:
# Split function stays as is
def split_by_app_user(df, train_users, val_users, test_users):
    train_df = df[df["torchserve_app_user"].isin(train_users)].copy()
    val_df   = df[df["torchserve_app_user"].isin(val_users)].copy()
    test_df  = df[df["torchserve_app_user"].isin(test_users)].copy()
    return train_df, val_df, test_df

# Calculate percentages
def calculate_split_percentages(train_df, val_df, test_df):
    total = len(train_df) + len(val_df) + len(test_df)
    return {
        "train_pct": len(train_df) / total * 100,
        "val_pct": len(val_df) / total * 100,
        "test_pct": len(test_df) / total * 100,
        "train_rows": len(train_df),
        "val_rows": len(val_df),
        "test_rows": len(test_df),
        "total_rows": total
    }

# Create X and y splits
def make_xy(df):
    X = df[[
        "torchserve_app_user",
        "node_id_src",
        "node_id_tgt",
        # all source metrics
        "torchserve_node_cpu_src",
        "torchserve_node_energy_src",
        "torchserve_node_power_src",
        "torchserve_app_cpu_src",
        "torchserve_app_energy_src",
        "torchserve_app_power_src",
        "torchserve_app_latency_src",
        "torchserve_app_qps_src"
    ]].copy()

    y = df[[
        "torchserve_node_cpu_tgt",
        "torchserve_node_energy_tgt",
        "torchserve_node_power_tgt",
        "torchserve_app_cpu_tgt",
        "torchserve_app_energy_tgt",
        "torchserve_app_power_tgt",
        "torchserve_app_latency_tgt",
        "torchserve_app_qps_tgt"
    ]].copy()
    return X, y

def shuffle_df(X, y, random_state=42):
    shuffled_idx = X.sample(frac=1, random_state=random_state).index
    return X.loc[shuffled_idx].reset_index(drop=True), y.loc[shuffled_idx].reset_index(drop=True)

def evaluate_model(model, X_val, y_val, plot=False):
    """
    Evaluate a multi-output regression model on validation data.

    Parameters
    ----------
    model : fitted sklearn-like regressor
    X_val : array-like
    y_val : pd.DataFrame
    plot : bool, optional

    Returns
    -------
    pd.DataFrame
    """

    targets = list(y_val.columns)
    y_pred = model.predict(X_val)

    if isinstance(y_pred, list):
        y_pred = np.column_stack(y_pred)

    results, r2_list, mae_list, rmse_list = [], [], [], []

    for i, target in enumerate(targets):
        y_true_t = y_val[target].values
        y_pred_t = y_pred[:, i]

        r2 = r2_score(y_true_t, y_pred_t)
        mae = mean_absolute_error(y_true_t, y_pred_t)
        rmse = mean_squared_error(y_true_t, y_pred_t, squared=False)

        results.append([target, r2, mae, rmse])
        r2_list.append(r2)
        mae_list.append(mae)
        rmse_list.append(rmse)

        if plot:
            fig, axes = plt.subplots(1, 2, figsize=(10, 4))
            fig.suptitle(target, fontsize=13)

            axes[0].scatter(y_true_t, y_pred_t, alpha=0.6)
            axes[0].plot([y_true_t.min(), y_true_t.max()],
                         [y_true_t.min(), y_true_t.max()], 'r--', lw=1)
            axes[0].set_xlabel("Actual")
            axes[0].set_ylabel("Predicted")
            axes[0].set_title("Predicted vs Actual")

            residuals = y_true_t - y_pred_t
            axes[1].scatter(y_pred_t, residuals, alpha=0.6)
            axes[1].axhline(0, color='red', linestyle='--', lw=1)
            axes[1].set_xlabel("Predicted")
            axes[1].set_ylabel("Residual")
            axes[1].set_title("Residuals vs Predicted")

            plt.tight_layout()
            plt.show()

    overall_r2_mean = np.mean(r2_list)
    overall_r2_global = r2_score(y_val, y_pred)
    overall_mae = np.mean(mae_list)
    overall_rmse = np.mean(rmse_list)

    results_df = pd.DataFrame(results, columns=["Target", "R2", "MAE", "RMSE"])
    overall_row = pd.DataFrame({
        "Target": ["Overall (mean)", "Overall (global)"],
        "R2": [overall_r2_mean, overall_r2_global],
        "MAE": [overall_mae, None],
        "RMSE": [overall_rmse, None]
    })
    results_df = pd.concat([results_df, overall_row], ignore_index=True)

    print(results_df)
    return results_df


In [4]:
# Apply

train_users = [1, 13, 25, 31, 43, 55]
val_users   = [7, 19]
test_users  = [37, 49]


train_df, val_df, test_df = split_by_app_user(df, train_users, val_users, test_users)

stats = calculate_split_percentages(train_df, val_df, test_df)
print(stats)

X_train, y_train = make_xy(train_df)
X_val, y_val     = make_xy(val_df)
X_test, y_test   = make_xy(test_df)

X_train, y_train = shuffle_df(X_train, y_train)
X_val, y_val     = shuffle_df(X_val, y_val)
X_test, y_test   = shuffle_df(X_test, y_test)

print("Shapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)


{'train_pct': 58.77706732577508, 'val_pct': 21.147247787406833, 'test_pct': 20.075684886818088, 'train_rows': 153146, 'val_rows': 55100, 'test_rows': 52308, 'total_rows': 260554}
Shapes:
X_train: (153146, 11) y_train: (153146, 8)
X_val: (55100, 11) y_val: (55100, 8)
X_test: (52308, 11) y_test: (52308, 8)


In [5]:
encode=1
model_name="random_forest"

# First model: Random forest

In [6]:
# rf =RandomForestRegressor(
#         n_estimators=200,
#         max_depth=None,
#         random_state=42,
#         n_jobs=-1
#     )

# rf.fit(X_train, y_train)

# print("Train R^2:", rf.score(X_train, y_train))
# print("Val R^2:", rf.score(X_val, y_val))


In [7]:

# # save the trained model
# joblib.dump(rf, f"./models/{model_name}_multioutput_scoredpairs.pkl")

# later, load it back
rf_loaded = joblib.load(f"./models/{model_name}_multioutput_scoredpairs.pkl")

# check it's working
print("Loaded model Val R^2:", rf_loaded.score(X_val, y_val))


Loaded model Val R^2: 0.8393735987692135


In [17]:
results_df = evaluate_model(rf_loaded, X_val, y_val, plot=False)


                       Target        R2         MAE        RMSE
0     torchserve_node_cpu_tgt  0.879076  246.770442  329.942019
1  torchserve_node_energy_tgt  0.992894  214.542408  335.907626
2   torchserve_node_power_tgt  0.988198    4.834819    7.109354
3      torchserve_app_cpu_tgt  0.892616  272.062274  356.213509
4   torchserve_app_energy_tgt  0.904240  238.149185  358.747759
5    torchserve_app_power_tgt  0.899360    4.574169    7.058053
6  torchserve_app_latency_tgt  0.332441   15.968049   23.495710
7      torchserve_app_qps_tgt  0.826164    4.552061    6.048523
8              Overall (mean)  0.839374  125.181676  178.065319
9            Overall (global)  0.839374         NaN         NaN


# XGBoosting

In [9]:
# Define model
base_xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    enable_categorical=True
)

# Wrap for multi-output regression
xgb_multi = MultiOutputRegressor(base_xgb)

# Fit
xgb_multi.fit(X_train, y_train)


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=True,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=300, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

In [10]:
results_df = evaluate_model(xgb_multi, X_val, y_val, plot=False)

                       Target        R2         MAE        RMSE
0     torchserve_node_cpu_tgt  0.494953  575.961474  674.290318
1  torchserve_node_energy_tgt  0.979595  471.488649  569.227830
2   torchserve_node_power_tgt  0.984029    7.072097    8.270269
3      torchserve_app_cpu_tgt  0.569301  630.890538  713.390083
4   torchserve_app_energy_tgt  0.887298  284.808128  389.190830
5    torchserve_app_power_tgt  0.858191    6.364366    8.378221
6  torchserve_app_latency_tgt  0.675685   12.987698   16.376736
7      torchserve_app_qps_tgt  0.504278    9.560554   10.214059
8              Overall (mean)  0.744166  249.891688  298.667293


# Neural Networks: MLP
### First Scaling and onehotencoding.

In [11]:
# Identify categorical and numeric features
cat_features = ["node_id_src", "node_id_tgt"]
num_features = [col for col in X_train.columns if col not in cat_features]

# --- 1. One-hot encode categorical features ---
# Fit only on train to avoid data leakage
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoder.fit(X_train[cat_features])

# Transform each split
X_train_cat = pd.DataFrame(
    encoder.transform(X_train[cat_features]),
    columns=encoder.get_feature_names_out(cat_features),
    index=X_train.index
)

X_val_cat = pd.DataFrame(
    encoder.transform(X_val[cat_features]),
    columns=encoder.get_feature_names_out(cat_features),
    index=X_val.index
)

X_test_cat = pd.DataFrame(
    encoder.transform(X_test[cat_features]),
    columns=encoder.get_feature_names_out(cat_features),
    index=X_test.index
)

# --- 2. Scale numeric features ---
x_scaler = MinMaxScaler()
x_scaler.fit(X_train[num_features])

X_train_num = pd.DataFrame(
    x_scaler.transform(X_train[num_features]),
    columns=num_features,
    index=X_train.index
)

X_val_num = pd.DataFrame(
    x_scaler.transform(X_val[num_features]),
    columns=num_features,
    index=X_val.index
)

X_test_num = pd.DataFrame(
    x_scaler.transform(X_test[num_features]),
    columns=num_features,
    index=X_test.index
)

# --- 3. Combine scaled numeric + encoded categorical ---
X_train_prepared = pd.concat([X_train_num, X_train_cat], axis=1)
X_val_prepared = pd.concat([X_val_num, X_val_cat], axis=1)
X_test_prepared = pd.concat([X_test_num, X_test_cat], axis=1)


# Model

In [22]:
# Define the MLP model
mlp = MLPRegressor(
    hidden_layer_sizes=(128, 64, 32),  # 3 hidden layers
    activation='relu',
    solver='adam',
    learning_rate_init=0.005,
    max_iter=1000,
    early_stopping=True,
    random_state=42,
    verbose=True
)
mlp.fit(X_train_prepared, y_train)

Iteration 1, loss = 768102.19462706
Validation score: 0.579441
Iteration 2, loss = 180164.01862859
Validation score: 0.866386
Iteration 3, loss = 42942.39641678
Validation score: 0.912455
Iteration 4, loss = 25869.25634597
Validation score: 0.933679
Iteration 5, loss = 18908.57672398
Validation score: 0.956929
Iteration 6, loss = 15375.37491847
Validation score: 0.967689
Iteration 7, loss = 13789.17596880
Validation score: 0.963835
Iteration 8, loss = 13093.20844088
Validation score: 0.968512
Iteration 9, loss = 12665.17971205
Validation score: 0.975451
Iteration 10, loss = 12447.08669129
Validation score: 0.979728
Iteration 11, loss = 12334.54710906
Validation score: 0.981092
Iteration 12, loss = 12004.80659052
Validation score: 0.980239
Iteration 13, loss = 11982.01887907
Validation score: 0.978746
Iteration 14, loss = 11741.08458764
Validation score: 0.964885
Iteration 15, loss = 11704.10884918
Validation score: 0.976413
Iteration 16, loss = 11614.51747598
Validation score: 0.975653

MLPRegressor(early_stopping=True, hidden_layer_sizes=(128, 64, 32),
             learning_rate_init=0.005, max_iter=1000, random_state=42,
             verbose=True)

In [26]:
results_df = evaluate_model(mlp, X_val_prepared, y_val, plot=False)

                       Target        R2         MAE        RMSE
0     torchserve_node_cpu_tgt  0.975790  108.554187  147.630252
1  torchserve_node_energy_tgt  0.998808   93.330054  137.606355
2   torchserve_node_power_tgt  0.997677    2.415919    3.154028
3      torchserve_app_cpu_tgt  0.983586  103.212113  139.268672
4   torchserve_app_energy_tgt  0.974583  133.752236  184.823212
5    torchserve_app_power_tgt  0.966447    3.001484    4.075375
6  torchserve_app_latency_tgt  0.611905   14.510211   17.914838
7      torchserve_app_qps_tgt  0.938404    2.904102    3.600452
8              Overall (mean)  0.930900   57.710038   79.759148
9            Overall (global)  0.930900         NaN         NaN


In [28]:
# # save the trained model
model_name = "mlp"
joblib.dump(mlp, f"./models/{model_name}_multioutput_scoredpairs_scaled_onehotencoded.pkl")

# later, load it back
mlp_loaded = joblib.load(f"./models/{model_name}_multioutput_scoredpairs_scaled_onehotencoded.pkl")

# check it's working
print("Loaded model Val R^2:", mlp_loaded.score(X_val_prepared, y_val))


Loaded model Val R^2: 0.9308998725764865


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [(128,64,32), (256,128,64), (128,128,64,32)],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.003, 0.005],
    'alpha': [0.0001, 0.001, 0.01]
}

mlp_base = MLPRegressor(max_iter=1000, early_stopping=True, random_state=42)

grid = GridSearchCV(mlp_base, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=2)
grid.fit(X_train_prepared, y_train)

print("Best parameters:", grid.best_params_)
print("Best CV R2:", grid.best_score_)


Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.003; total time= 4.0min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.005; total time= 5.7min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.005; total time= 5.8min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.005; total time= 5.9min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.003; total time= 6.9min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001; total time= 7.2min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(256, 128, 64), learning_rate_init=0.003; total time= 4.6min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.003; tota